In [1]:
# Author: Sunny You
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
os.chdir('/Users/sunnyyou/Documents/Real_Time_HAI/HPCNN/IMERG')

In [35]:
v = pd.read_csv("ATL_OCEAN_SHIPS.csv")
v = v[["GIS_ID", "VMAX"]]
v.VMAX -= v.VMAX.min()
v.VMAX.max()

140

In [3]:
a = pd.read_table("IMERG_CSV/ATL_200001_TD_2000060718.csv", header = None, sep = ',', engine = "python")

b = a[30:91]
print(b.shape)
b = b.iloc[:, 30:91]
print(b.shape)
b

(61, 121)
(61, 61)


,30,31,32,33,34,35,36,37,38,39,...,81,82,83,84,85,86,87,88,89,90
30,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,1.626409e-03,1.626409e-03,2.168545e-03,1.084273e-02,1.084273e-02
31,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1.626409e-03,2.406105e-02,1.533615e-01,1.286941e-01
32,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.849342e-01,1.130242e+00,1.038988e+00
33,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,1.093254e-01,0.033496,0.027357,0.033639,3.510619e-02,3.745444e-02,2.489135e-02,2.758152e-01,1.659571e+00,1.976249e+00
34,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,7.551283e-01,0.114707,0.122108,0.070680,2.206774e-01,8.354013e-02,2.226315e-01,3.256500e-01,7.839627e-01,1.101383e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,1.541524e-08,2.837465e-03,1.344267e-02,...,5.398957e-04,0.000000,0.001098,0.000000,3.703402e-08,9.621747e-08,1.813826e-07,3.997430e-07,3.831069e-07,6.576519e-07
87,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,1.805897e-08,1.136716e-08,...,7.954760e-04,0.006481,0.033222,0.005629,7.732282e-08,8.982069e-08,4.971269e-03,6.285593e-02,1.859253e-01,5.111344e-02
88,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,6.446066e-08,0.541343,2.959741,1.702892,2.524809e-01,4.052978e-02,1.120260e-01,3.815346e-01,1.084481e+00,5.904219e-01
89,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,2.221922e-03,2.891982,16.368359,9.863838,1.555263e+00,2.512188e-01,4.860221e-01,2.087434e-01,1.143255e+00,4.631010e+00


In [6]:
train = pd.read_csv("IC_Data/ATL/P24_train_resample.csv")
train = train[["GIS_ID", "DATE", "SHIPS_PER", "SHIPS_POT", "VMAX", "SHDC", "IC", "Category"]]
# train = train.drop(["VMAX_FT"], axis = 1)
train.columns = ["GIS_ID", "DATE", "PER", "POT", "VMAX", "SHDC_FT", "IC", "Category"]
train

,GIS_ID,DATE,PER,POT,VMAX,SHDC_FT,IC,Category
0,ATL_200002_TD_2000062318,2000062318,9.310000e-09,84.0,30,10.100000,0,TD
1,ATL_200002_TD_2000062400,2000062400,0.000000e+00,84.0,30,4.300000,0,TD
2,ATL_200002_TD_2000062406,2000062406,0.000000e+00,84.0,30,4.400000,0,TD
3,ATL_200002_TD_2000062412,2000062412,0.000000e+00,85.0,30,5.700000,0,TD
4,ATL_200002_TD_2000062418,2000062418,0.000000e+00,87.0,30,6.200000,-5,TD
...,...,...,...,...,...,...,...,...
7411,ATL_201913_C4_2019092712,2019092712,-5.000000e+00,6.0,120,18.800001,-15,Maj
7412,ATL_201905_C3_2019090312,2019090312,-1.500000e+01,33.0,100,8.400001,-10,Maj
7413,ATL_201408_C4_2014101500,2014101500,2.000000e+01,39.0,115,6.700000,-5,Maj
7414,ATL_201511_C4_2015100212,2015100212,-5.000000e+00,40.0,115,25.500000,20,Maj


In [38]:
# test = pd.read_csv("IC_Data/P18_2018_test.csv")
# test = test[["GIS_ID", "DATE", "SHIPS_PER", "SHIPS_POT", "VMAX", "SHDC", "IC", "Category"]]
# test = test.drop(["VMAX_FT"], axis = 1)
# test.columns = ["GIS_ID", "DATE", "PER", "POT", "VMAX", "SHDC_FT", "IC", "Category"]
# test

,GIS_ID,DATE,PER,POT,VMAX,SHDC_FT,IC,Category
0,ATL_201801_TS_2018052800,2018052800,15.0,67.0,55,20.000000,-10,TS
1,ATL_201802_TS_2018070506,2018070506,5.0,52.0,35,6.000000,20,TS
2,ATL_201802_TS_2018070512,2018070512,5.0,51.0,40,4.900000,30,TS
3,ATL_201802_TS_2018070518,2018070518,10.0,48.0,45,8.100000,25,TS
4,ATL_201802_TS_2018070600,2018070600,15.0,41.0,55,7.000000,10,TS
...,...,...,...,...,...,...,...,...
380,ATL_201920_TS_2019112206,2019112206,0.0,56.0,50,40.200001,10,TS
381,ATL_201920_TS_2019112212,2019112212,0.0,52.0,50,41.400002,10,TS
382,ATL_201920_TS_2019112218,2019112218,5.0,38.0,55,42.600002,0,TS
383,ATL_201920_TS_2019112300,2019112300,10.0,24.0,60,33.600002,-5,TS


In [7]:
# train = pd.concat([train, test])
test = pd.DataFrame(np.reshape(['ATL_202313_TS_2023090606', '2023090606', 20, 102, 55, 13.0, -999, 'TS'], (1, 8)))
test.columns = ('GIS_ID', 'DATE', 'PER', 'POT', 'VMAX', 'SHDC_FT', 'IC', 'Category')
train = train.iloc[:, :-1]
train = train.reset_index(drop = False)
print(train.head(), train.shape)
print(test.head(), test.shape)

   index                    GIS_ID        DATE           PER   POT  VMAX   
0      0  ATL_200002_TD_2000062318  2000062318  9.310000e-09  84.0    30  \
1      1  ATL_200002_TD_2000062400  2000062400  0.000000e+00  84.0    30   
2      2  ATL_200002_TD_2000062406  2000062406  0.000000e+00  84.0    30   
3      3  ATL_200002_TD_2000062412  2000062412  0.000000e+00  85.0    30   
4      4  ATL_200002_TD_2000062418  2000062418  0.000000e+00  87.0    30   

   SHDC_FT  IC  
0     10.1   0  
1      4.3   0  
2      4.4   0  
3      5.7   0  
4      6.2  -5   (7416, 8)
                     GIS_ID        DATE PER  POT VMAX SHDC_FT    IC Category
0  ATL_202313_TS_2023090606  2023090606  20  102   55    13.0  -999       TS (1, 8)


In [8]:
train_img = []
train_ships = []
train_label = []
test_img = []
test_ships = []
test_label = []
for f in range(len(train.GIS_ID)) :
    filename = "IMERG_CSV/" + train.GIS_ID[f] + ".csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        train_img.append(temp)
        lab = train.IC[f]
        train_label.append(lab)
        ships = np.array([train.VMAX[f], train.POT[f], train.PER[f], train.SHDC_FT[f]])
        train_ships.append(ships)
    except Exception as e:
        pass

for f in range(len(test.GIS_ID)) :
    filename = "IMERG_CSV/" + test.GIS_ID[f] + ".csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        test_img.append(temp)
        lab = test.IC[f]
        test_label.append(lab)
        ships = np.array([test.VMAX[f], test.POT[f], test.PER[f], test.SHDC_FT[f]])
        test_ships.append(ships)
    except Exception as e:
        pass

In [9]:
print(len(train_img))
print(len(train_ships))
print(len(train_label))
print(len(test_img))
print(len(test_ships))
print(len(test_label))

7416
7416
7416
1
1
1


In [10]:
test_ships = np.float64(test_ships)
test_label = np.float64(test_label)

In [11]:
type(test_ships[0][0])

numpy.float64

In [12]:
X_train_img = train_img
X_train_ships = train_ships
y_train = train_label
X_test_img = test_img
X_test_ships = test_ships
y_test = test_label

In [13]:
X_train_img = np.array(X_train_img)
X_train_img = X_train_img.reshape(-1,41,41,1)
X_train_img = X_train_img.astype('float32')
X_train_ships = np.array(X_train_ships)
X_train_ships = X_train_ships.reshape(-1,4)
y_train = np.array(y_train)

In [14]:
X_test_img = np.array(X_test_img)
X_test_img = X_test_img.reshape(-1,41,41,1)
X_test_img = X_test_img.astype('float32')
X_test_ships = np.array(X_test_ships)
X_test_ships = X_test_ships.reshape(-1,4)
y_test = np.array(y_test)

In [15]:
import tensorflow as tf 
from tensorflow import keras

2023-09-06 10:37:55.632686: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
# model1 = keras.Sequential([
#    keras.layers.Conv2D(64,5, activation = 'relu', input_shape = (24,24,1)),
#    keras.layers.Conv2D(64,5, activation = 'relu'),
#    keras.layers.Conv2D(64,1, activation = 'relu'),
#    keras.layers.MaxPool2D((2,2), strides = 2),
#    keras.layers.BatchNormalization(),
#    keras.layers.Conv2D(64, 4, activation='relu'),
#    keras.layers.Conv2D(64, 4, activation='relu'),
#    keras.layers.Conv2D(256,1, activation = 'relu'),
#    keras.layers.BatchNormalization(),
#    keras.layers.Flatten(),
#    keras.layers.Dense(256),
#    keras.layers.Dense(165)
# ])

# model2 = keras.Sequential([
#    keras.layers.Conv2D(256, 5, activation='relu', input_shape=(24,24,1)),
#    keras.layers.MaxPool2D(2,2),
#    keras.layers.BatchNormalization(),
#    keras.layers.Conv2D(128, 1, activation = 'relu'),
#    keras.layers.Conv2D(128,3, activation = 'relu'), 
#    keras.layers.MaxPool2D(2,2),
#    keras.layers.BatchNormalization(),
#    keras.layers.Conv2D(64, 1, activation = 'relu'),
#    keras.layers.Conv2D(64,2, activation = 'relu'), 
#    keras.layers.MaxPool2D(2,2),
#    keras.layers.BatchNormalization(),
#    keras.layers.Flatten(),
#    keras.layers.Dense(165, activation = 'linear')
# ])
# model3 = keras.Sequential([
#     keras.layers.Conv2D(256, (5,2), activation='relu', input_shape=(24,24,1)),
#     keras.layers.Conv2D(256, (2,5), activation = 'relu'),
#     keras.layers.MaxPool2D(2,2),
#     keras.layers.BatchNormalization(),
#     keras.layers.Conv2D(128,3, activation = 'relu'), 
#     keras.layers.MaxPool2D(2,2),
#     keras.layers.BatchNormalization(),
#     keras.layers.Conv2D(64,2, activation = 'relu'), 
#     keras.layers.MaxPool2D(2,2),
#     keras.layers.BatchNormalization(),
#     keras.layers.Flatten(),
#     keras.layers.Dense(165)
# ])
# model4 = keras.Sequential([
#     keras.layers.Conv2D(256, (5,5), activation = 'relu', input_shape = (24,24,1)), 
#     keras.layers.MaxPool2D(2,2), 
#     keras.layers.Flatten(), 
#     keras.layers.Dense(165)
# ])


# model1.summary()
# model2.summary()
# model3.summary()
# model4.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 20, 20, 64)        1664      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 64)        102464    
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 64)        4160      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 5, 5, 64)          65600     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 2, 2, 64)          6

In [530]:
#300km
ships_input = keras.Input(shape =(2,), name = "ships_layer")
img_input = keras.Input(shape =(61, 61, 1), name = "img_layer")

w = keras.layers.Conv2D(64,12) (img_input)
w = keras.layers.Conv2D(64,12)(w)
w = keras.layers.Conv2D(64,2)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.relu(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 9)(w)
w = keras.layers.Conv2D(64, 9)(w)
w = keras.layers.Conv2D(256,2)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, ships_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(165)(output_layer1)

new_model1 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 12) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,2, activation = 'relu')(x)
x = keras.layers.Conv2D(128,7)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,2, activation = 'relu')(x)
x = keras.layers.Conv2D(64,4)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, ships_input])
output_layer2 = keras.layers.Dense(165)(merged_model2)

new_model2 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (10,4))(img_input)
y = keras.layers.Conv2D(256, (4,10))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,6)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,4)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, ships_input])
output_layer3 = keras.layers.Dense(165)(merged_model3)

new_model3 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

z = keras.layers.Conv2D(256, 10)(img_input)
z = keras.layers.MaxPool2D(2,2)(z)
img_output4 = keras.layers.Flatten()(z)

merged_model4 = keras.layers.concatenate([img_output4, ships_input])
output_layer4 = keras.layers.Dense(165)(merged_model4)

new_model4 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer4, name = "model_4")

new_model4.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_layer (InputLayer)          [(None, 61, 61, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_800 (Conv2D)             (None, 50, 50, 64)   9280        img_layer[0][0]                  
__________________________________________________________________________________________________
conv2d_801 (Conv2D)             (None, 39, 39, 64)   589888      conv2d_800[0][0]                 
__________________________________________________________________________________________________
conv2d_802 (Conv2D)             (None, 38, 38, 64)   16448       conv2d_801[0][0]                 
____________________________________________________________________________________________

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_layer (InputLayer)          [(None, 61, 61, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_815 (Conv2D)             (None, 52, 52, 256)  25856       img_layer[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_407 (MaxPooling2D (None, 26, 26, 256)  0           conv2d_815[0][0]                 
__________________________________________________________________________________________________
flatten_203 (Flatten)           (None, 173056)       0           max_pooling2d_407[0][0]          
____________________________________________________________________________________________

In [16]:
#200km
ships_input = keras.Input(shape =(4,), name = "ships_layer")
img_input = keras.Input(shape =(41, 41, 1), name = "img_layer")

w = keras.layers.Conv2D(64,8) (img_input)
w = keras.layers.Conv2D(64,8)(w)
w = keras.layers.Conv2D(64,1)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.relu(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(256,1)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, ships_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(165)(output_layer1)

new_model1 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 8) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,1, activation = 'relu')(x)
x = keras.layers.Conv2D(128,5)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,1, activation = 'relu')(x)
x = keras.layers.Conv2D(64,3)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, ships_input])
output_layer2 = keras.layers.Dense(165)(merged_model2)

new_model2 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (6,2))(img_input)
y = keras.layers.Conv2D(256, (2,6))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,4)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,3)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, ships_input])
output_layer3 = keras.layers.Dense(165)(merged_model3)

new_model3 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

z = keras.layers.Conv2D(256, 7)(img_input)
z = keras.layers.MaxPool2D(2,2)(z)
img_output4 = keras.layers.Flatten()(z)

merged_model4 = keras.layers.concatenate([img_output4, ships_input])
output_layer4 = keras.layers.Dense(165)(merged_model4)

new_model4 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer4, name = "model_4")

new_model4.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_layer (InputLayer)         [(None, 41, 41, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 34, 34, 64)   4160        ['img_layer[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)              (None, 27, 27, 64)   262208      ['conv2d[0][0]']                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 27, 27, 64)   4160        ['conv2d_1[0][0]']               
                                                                                            

In [357]:
#150km
ships_input = keras.Input(shape =(2,), name = "ships_layer")
img_input = keras.Input(shape =(31, 31, 1), name = "img_layer")

w = keras.layers.Conv2D(64,6) (img_input)
w = keras.layers.Conv2D(64,6)(w)
w = keras.layers.Conv2D(64,1)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.relu(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 5)(w)
w = keras.layers.Conv2D(64, 5)(w)
w = keras.layers.Conv2D(256,1)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, ships_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(165)(output_layer1)

new_model1 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 6) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,1, activation = 'relu')(x)
x = keras.layers.Conv2D(128,4)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,1, activation = 'relu')(x)
x = keras.layers.Conv2D(64,2)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, ships_input])
output_layer2 = keras.layers.Dense(165)(merged_model2)

new_model2 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (5,2))(img_input)
y = keras.layers.Conv2D(256, (2,5))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,3)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,2)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, ships_input])
output_layer3 = keras.layers.Dense(165)(merged_model3)

new_model3 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

z = keras.layers.Conv2D(256, 5)(img_input)
z = keras.layers.MaxPool2D(2,2)(z)
img_output4 = keras.layers.Flatten()(z)

merged_model4 = keras.layers.concatenate([img_output4, ships_input])
output_layer4 = keras.layers.Dense(165)(merged_model4)

new_model4 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer4, name = "model_4")

new_model4.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_layer (InputLayer)          [(None, 31, 31, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_576 (Conv2D)             (None, 26, 26, 64)   2368        img_layer[0][0]                  
__________________________________________________________________________________________________
conv2d_577 (Conv2D)             (None, 21, 21, 64)   147520      conv2d_576[0][0]                 
__________________________________________________________________________________________________
conv2d_578 (Conv2D)             (None, 21, 21, 64)   4160        conv2d_577[0][0]                 
____________________________________________________________________________________________

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_layer (InputLayer)          [(None, 31, 31, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_591 (Conv2D)             (None, 27, 27, 256)  6656        img_layer[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_295 (MaxPooling2D (None, 13, 13, 256)  0           conv2d_591[0][0]                 
__________________________________________________________________________________________________
flatten_147 (Flatten)           (None, 43264)        0           max_pooling2d_295[0][0]          
____________________________________________________________________________________________

In [188]:
#100km
ships_input = keras.Input(shape =(2), name = "ships_layer")
img_input = keras.Input(shape =(21,21,1), name = "img_layer")

w = keras.layers.Conv2D(64,4) (img_input)
w = keras.layers.Conv2D(64,4)(w)
w = keras.layers.Conv2D(64,1)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.relu(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(256,1)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, ships_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(165)(output_layer1)

new_model1 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 4) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,1, activation = 'relu')(x)
x = keras.layers.Conv2D(128,2)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,1, activation = 'relu')(x)
x = keras.layers.Conv2D(64,1)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, ships_input])
output_layer2 = keras.layers.Dense(165)(merged_model2)

new_model2 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (3,1))(img_input)
y = keras.layers.Conv2D(256, (1,3))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,2)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,1)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, ships_input])
output_layer3 = keras.layers.Dense(165)(merged_model3)

new_model3 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

z = keras.layers.Conv2D(256, 3)(img_input)
z = keras.layers.MaxPool2D(2,2)(z)
img_output4 = keras.layers.Flatten()(z)

merged_model4 = keras.layers.concatenate([img_output4, ships_input])
output_layer4 = keras.layers.Dense(165)(merged_model4)

new_model4 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer4, name = "model_4")

new_model4.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_layer (InputLayer)          [(None, 21, 21, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_272 (Conv2D)             (None, 18, 18, 64)   1088        img_layer[0][0]                  
__________________________________________________________________________________________________
conv2d_273 (Conv2D)             (None, 15, 15, 64)   65600       conv2d_272[0][0]                 
__________________________________________________________________________________________________
conv2d_274 (Conv2D)             (None, 15, 15, 64)   4160        conv2d_273[0][0]                 
____________________________________________________________________________________________

In [17]:
new_model1.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model2.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model3.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model4.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])

In [17]:
# model3.fit(X_train, y_train, epochs=32, batch_size=32, validation_split = 0.1)
# res = model3.evaluate(X_test, y_test)
# print("MAE = " + str(res[0]))
# print("RMSE = " + str((res[2]) ** 0.5))

Epoch 1/32
833/833 [==============================] - 133s 159ms/step - loss: 68.3868 - mae: 68.3868 - mse: 6236.8566 - val_loss: 48.6292 - val_mae: 48.6292 - val_mse: 2582.8438
Epoch 2/32
833/833 [==============================] - 133s 160ms/step - loss: 15.2440 - mae: 15.2440 - mse: 392.0005 - val_loss: 46.3273 - val_mae: 46.3273 - val_mse: 2346.1006
Epoch 3/32
833/833 [==============================] - 133s 159ms/step - loss: 11.2150 - mae: 11.2150 - mse: 224.8393 - val_loss: 49.8577 - val_mae: 49.8577 - val_mse: 2721.2852
Epoch 4/32
833/833 [==============================] - 133s 159ms/step - loss: 9.4213 - mae: 9.4213 - mse: 156.3978 - val_loss: 44.5311 - val_mae: 44.5311 - val_mse: 2261.5693
Epoch 5/32
833/833 [==============================] - 133s 159ms/step - loss: 8.5597 - mae: 8.5597 - mse: 128.1547 - val_loss: 50.4979 - val_mae: 50.4979 - val_mse: 2870.5139
Epoch 6/32
833/833 [==============================] - 132s 159ms/step - loss: 7.9717 - mae: 7.9717 - mse: 109.2566 - v

In [18]:
new_model3.fit([X_train_img, X_train_ships], y_train, epochs=3, batch_size=8, validation_split = 0.1)
res = new_model3.evaluate([X_test_img, X_test_ships], y_test)
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))

Epoch 1/3
835/835 [==============================] - 181s 214ms/step - loss: 11.2496 - mae: 11.2496 - mse: 229.3993 - val_loss: 13.0121 - val_mae: 13.0121 - val_mse: 282.5752
Epoch 2/3
835/835 [==============================] - 178s 213ms/step - loss: 10.3442 - mae: 10.3442 - mse: 197.1038 - val_loss: 12.9410 - val_mae: 12.9410 - val_mse: 273.8423
Epoch 3/3
1/1 [==============================] - 0s 219ms/step - loss: 1014.0851 - mae: 1014.0851 - mse: 1028369.0625
MAE = 1014.0851440429688
RMSE = 1014.0853329478738


In [19]:
preds = new_model3.predict([X_test_img, X_test_ships])
a = np.average(preds, axis = 1)
test['preds'] = a

1/1 [==============================] - 0s 160ms/step


In [20]:
test

,GIS_ID,DATE,PER,POT,VMAX,SHDC_FT,IC,Category,preds
0,ATL_202313_TS_2023090606,2023090606,20,102,55,13.0,-999,TS,15.085094


In [23]:
test.to_csv("/Users/hajian/Documents/sunny/palmetto_high/Grade_10/Sci_Fair_2022/Output/Predictions/ATL_24h_Model3_200km_Predictions_2018_New.csv")

In [138]:
x

array([ 46.885475,  49.229416,  53.542645,  53.056805,  53.19665 ,
        43.343914,  42.296223,  42.46807 ,  47.794773,  67.17186 ,
        61.718346,  59.369804,  53.701538,  55.901005,  57.23753 ,
        48.804035,  51.86434 ,  46.99658 ,  52.867046,  49.755127,
        47.766068,  47.216316,  46.178207,  47.524982,  52.433357,
        64.225296,  69.84559 ,  72.28777 ,  38.14173 ,  29.842146,
        27.81076 ,  32.13262 ,  34.317207,  40.552475,  24.972866,
        26.924538,  26.79639 ,  25.229164,  21.103916,  22.758099,
        23.571857,  22.074652,  25.429724,  24.825964,  25.180311,
        23.676407,  24.811031,  25.293427,  25.05794 ,  25.813631,
        25.843685,  28.660997,  29.890465,  29.137138,  25.47519 ,
        25.071106,  26.02442 ,  32.03169 ,  47.15371 ,  25.729773,
        24.703226,  31.519093,  33.391663,  37.744358,  36.77495 ,
        44.190613,  40.111168,  52.18968 ,  58.659286,  57.66392 ,
        65.366745,  70.81807 ,  50.07538 ,  33.95543 ,  33.887

In [142]:
test['model4_preds'] = a

,Unnamed: 0,GIS_ID,BASIN,DATE,JULDAY,LATITUDE,LONGITUDE,MSLP,NAME,STORM_ID,CAT,LAND,VMAX,VMAX_N06H,VMAX_N12H,file,model4_preds,model1_preds,model2_preds,model3_preds
0,6198,ATL_201601_TS_2016010712,ATL,2016010712,2457395.00,28.7,-73.8,997,ALEX,201601,TS,400,50,45,40,ATL_201601_TS_2016010712_400.csv,46.885475,50.177475,47.781120,44.487297
1,6199,ATL_201601_TS_2016010718,ATL,2016010718,2457395.25,30.0,-72.5,987,ALEX,201601,TS,700,55,50,45,ATL_201601_TS_2016010718_700.csv,49.229416,49.576599,43.861824,44.414146
2,6200,ATL_201601_TS_2016010800,ATL,2016010800,2457395.50,31.4,-70.6,986,ALEX,201601,TS,700,55,55,50,ATL_201601_TS_2016010800_700.csv,53.542645,59.449253,52.837536,54.962795
3,6201,ATL_201601_TS_2016010806,ATL,2016010806,2457395.75,32.4,-68.8,986,ALEX,201601,TS,400,55,55,55,ATL_201601_TS_2016010806_400.csv,53.056805,53.845360,49.648674,52.348698
4,6202,ATL_201601_TS_2016010812,ATL,2016010812,2457396.00,33.0,-67.1,991,ALEX,201601,TS,400,45,55,55,ATL_201601_TS_2016010812_400.csv,53.196651,55.642189,55.389004,57.213150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,6586,ATL_201616_TS_2016112512,ATL,2016112512,2457718.00,10.3,-87.5,995,OTTO,201616,TS,400,55,60,65,ATL_201616_TS_2016112512_400.csv,59.663162,58.260784,61.240955,61.270626
367,6587,ATL_201616_TS_2016112518,ATL,2016112518,2457718.25,10.0,-88.8,997,OTTO,201616,TS,400,50,55,60,ATL_201616_TS_2016112518_400.csv,53.706024,52.891582,53.086079,52.610256
368,6588,ATL_201616_TS_2016112600,ATL,2016112600,2457718.50,9.7,-90.2,1000,OTTO,201616,TS,700,45,50,55,ATL_201616_TS_2016112600_700.csv,53.244343,50.944939,51.578381,47.597439
369,6589,ATL_201616_TS_2016112606,ATL,2016112606,2457718.75,9.4,-91.7,1003,OTTO,201616,TS,700,40,45,50,ATL_201616_TS_2016112606_700.csv,43.265495,43.461235,46.139259,41.228050


In [143]:
np.mean(abs(test.VMAX-test.model4_preds))

50.8733153638814

In [103]:
tf.__version__

'2.4.0'